In [ ]:
import os
import shutil
from pathlib import Path
import re

def organize_fits_files(source_dir, destination_dir):
    """
    Organize FITS files from source directory into groups of 4 (w1-w4) in destination directory.
    
    Args:
        source_dir (str): Path to source directory containing FITS files
        destination_dir (str): Path to destination directory where organized files will be stored
    """
    # Create destination directory if it doesn't exist
    Path(destination_dir).mkdir(parents=True, exist_ok=True)
    
    # Dictionary to store groups of related files
    file_groups = {}
    
    # Walk through all subdirectories
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.fits'):
                # Extract the base name without w1/w2/w3/w4
                match = re.search(r'(.*?)(w[1-4])(.*?\.fits)', file)
                if match:
                    base_name = match.group(1) + match.group(3)
                    band = match.group(2)
                    
                    # Initialize group if not exists
                    if base_name not in file_groups:
                        file_groups[base_name] = {'files': {}, 'source_path': root}
                    
                    # Store the file path for this band
                    file_groups[base_name]['files'][band] = file

    # Process complete groups
    for base_name, group_info in file_groups.items():
        files = group_info['files']
        source_path = group_info['source_path']
        
        # Check if we have all 4 bands
        if len(files) == 4 and all(f'w{i}' in files for i in range(1, 5)):
            # Create a new folder for this group
            group_folder = os.path.join(destination_dir, base_name.replace('.fits', ''))
            Path(group_folder).mkdir(exist_ok=True)
            
            # Copy files to new location
            for band, filename in files.items():
                source = os.path.join(source_path, filename)
                destination = os.path.join(group_folder, filename)
                shutil.copy2(source, destination)
            
            print(f"Organized group: {base_name}")
        else:
            print(f"Incomplete group {base_name}: missing bands {[f'w{i}' for i in range(1, 5) if f'w{i}' not in files]}")

    print("\nOrganization complete!")
    print(f"Total complete groups organized: {sum(1 for group in file_groups.values() if len(group['files']) == 4)}")
    print(f"Total incomplete groups skipped: {sum(1 for group in file_groups.values() if len(group['files']) != 4)}")

if __name__ == "__main__":
    # Example usage
    source_directory = "./Arbitrary"
    destination_directory = "./test_organize"
    
    organize_fits_files(source_directory, destination_directory)

In [ ]:
#TESTING ONLY

root_folder = "./test_organize"

# Loop through each subfolder and its files
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)
    if os.path.isdir(subfolder_path):  # Ensure it's a subfolder
        print(f"Processing subfolder: {subfolder_path}")
        files = []
        for file in os.listdir(subfolder_path):
            print(file)
            files.append(file)
            file_path = os.path.join(subfolder_path, file)
            #if os.path.isfile(file_path):  # Ensure it's a file
                #print(f"  Found file: {file_path}")
    if (len(files) != 4):
        print("ERROR, MORE THAN 4 FILES in folder")
    checking = []
    for i in range(len(files)):
        checking.append(files[i][:14] + files[i][16:])
    for c in checking:
        for i in range(len(checking)):
            if (c != checking[i]):
                print("ERROR: " + c + "   " + checking[i])
